<a href="https://colab.research.google.com/github/SteenJennings/Neural-Net-Options/blob/master/Kevin/ROTS_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# AlphaVantage CSV Processing

In [101]:
pip install alpha_vantage

Using several API keys to pull the technical data

In [102]:
from alpha_vantage.timeseries import  TimeSeries
from alpha_vantage.techindicators import TechIndicators
import pandas as pd
import time

tickers = ['AMD', 'SPY', 'TSLA', 'AAPL', 'AMZN', 'OPEN', 'SNAP', 'GOOG', 'NVDA', 'FSLY', 'ROKU', 'CHWY', 'TQQQ']
#ticker = tickers[8]
ticker = 'DIS'

#update the ti for API call limit
API_key = '04ZKSL2N50GIBN85'    #steens key
ti = TechIndicators(key = API_key, output_format='pandas')

# RSI/EMA data
data_rsi15 = ti.get_rsi(ticker, interval='daily',time_period=15, series_type='close')
data_ema5 = ti.get_ema(ticker,interval='daily',time_period=5, series_type='close')
data_ema10 = ti.get_ema(ticker,interval='daily',time_period=10, series_type='close')
data_ema20 = ti.get_ema(ticker,interval='daily',time_period=20, series_type='close')
data_ema125 = ti.get_ema(ticker,interval='daily',time_period=125, series_type='close')

#update the ti for API call limit
API_key = 'D6XW69E9UXBCXJBK'     #steen@yopmail.com
ti = TechIndicators(key = API_key, output_format='pandas')

# ADX data
data_adx5 = ti.get_adx(ticker,interval='daily',time_period=5)
data_adx10 = ti.get_adx(ticker,interval='daily',time_period=10)
data_adx20 = ti.get_adx(ticker,interval='daily',time_period=15)

#update the ti for API call limit
API_key = '50F3VWQLVV9C4HGT'    #kevins key
ti = TechIndicators(key = API_key, output_format='pandas')

# Bolinger Bands data
data_bb5 = ti.get_bbands(ticker,interval='daily',time_period=5, series_type='close')
data_bb10 = ti.get_bbands(ticker,interval='daily',time_period=10, series_type='close')
data_bb20 = ti.get_bbands(ticker,interval='daily',time_period=20, series_type='close')

# grabs the stock price data
API_key = "PK3E26W0JBG0A8LI"  # steen other email key
ts = TimeSeries(key = API_key, output_format='pandas')

# make sure that the call limit has not been met
data = ts.get_daily_adjusted(ticker, outputsize='full')
# inverts the DF to fit technical format
data = data[0].iloc[::-1]
# trims to match size of other df
data = data.iloc[149:]
# this commented section is used to trim the data after market close when the technical data is not updated but time series data is updated (1-4pm or so)
# data = data[:-1]
# data
pct_price = data[['5. adjusted close']]
pct_price = pct_price.pct_change()

Trimming the data and adding to a single dataframe

In [103]:
# this code trims the data for each indicator type to fit into one dataframe, with ema5 heading the list
ema5_df = data_ema5[0]
ema5_df = ema5_df.iloc[145:]
ema10_df = data_ema10[0]
ema10_df = ema10_df.iloc[140:]
ema20_df = data_ema20[0]
ema20_df = ema20_df.iloc[130:]
ema125_df = data_ema125[0]
ema125_df = ema125_df.iloc[25:]
rsi15_df = data_rsi15[0]
rsi15_df = rsi15_df.iloc[134:]
adx5_df = data_adx5[0]
adx5_df = adx5_df.iloc[140:]
adx10_df = data_adx10[0]
adx10_df = adx10_df.iloc[130:]
adx20_df = data_adx20[0]
adx20_df = adx20_df.iloc[120:]
# converting the data type of the bb DF's and flipping the orientation
bb5_df = data_bb5[0].iloc[::-1]
bb10_df = data_bb10[0].iloc[::-1]
bb20_df = data_bb20[0].iloc[::-1]
# resizing the data
bb5_df = bb5_df.iloc[145:]
bb10_df = bb10_df.iloc[140:]
bb20_df = bb20_df.iloc[130:]
print(bb20_df)

#this is used to dynamically merge all dataframes using it as an element of the below loop
df_list = []

# storing everything in a tuple so we can access the column name and the DF itself
ema5_df = (ema5_df,"EMA_5")
ema10_df = (ema10_df,"EMA_10")
ema20_df = (ema20_df,"EMA_20")
ema125_df = (ema125_df,"EMA_125")
adx5_df = (adx5_df,"ADX_5")
adx10_df = (adx10_df,"ADX_10")
adx20_df = (adx20_df,"ADX_20")
bb5_df = (bb5_df,"BB_5")
bb10_df = (bb10_df,"BB_10")
bb20_df = (bb20_df,"BB_20")
rsi15_df = (rsi15_df,"RSI_15")

#appending the items to our list
df_list.append(ema5_df)
df_list.append(ema10_df)
df_list.append(ema20_df)
df_list.append(ema125_df)
df_list.append(adx5_df)
df_list.append(adx10_df)
df_list.append(adx20_df)
df_list.append(bb5_df)
df_list.append(bb10_df)
df_list.append(bb20_df)
df_list.append(rsi15_df)

#iterating across our list to fill our DF with our trimmed values
for dataframe_tuple in df_list:
  cur_dataframe, indicator_type = dataframe_tuple
  if indicator_type[0:3] == "EMA":
    temp = "EMA"
  elif indicator_type[0:3] == "RSI":
    temp = "RSI"
  elif indicator_type[0:3] == "ADX":  
    temp = "ADX"
  # this indicator has three elements. Might connect these.
  elif indicator_type[0:2] == "BB":  
    temp_list = list(cur_dataframe["Real Upper Band"])
    data[indicator_type + ' ' + 'Upper Band'] = temp_list
    continue
  else:
    temp = indicator_type
  temp_list = list(cur_dataframe[temp])
  data[indicator_type] = temp_list

data


            Real Middle Band  Real Upper Band  Real Lower Band
date                                                          
2000-06-05           31.8283          33.0189          30.6378
2000-06-06           31.8408          33.0243          30.6572
2000-06-07           31.8190          33.0285          30.6095
2000-06-08           31.7949          33.0135          30.5763
2000-06-09           31.8116          33.0494          30.5739
...                      ...              ...              ...
2021-05-24          179.0120         190.9171         167.1069
2021-05-25          178.5885         190.2630         166.9140
2021-05-26          178.2425         189.7361         166.7489
2021-05-27          177.9280         188.9637         166.8923
2021-05-28          177.5595         187.9639         167.1551

[5280 rows x 3 columns]


,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,EMA_5,EMA_10,EMA_20,EMA_125,ADX_5,ADX_10,ADX_20,BB_5 Upper Band,BB_10 Upper Band,BB_20 Upper Band,RSI_15
date,,,,,,,,,,,,,,,,,,,
2000-06-05,40.50,41.00,40.00,40.63,31.598527,4220200.0,0.0,1.0,31.7931,31.7974,31.7960,28.0832,25.1923,13.3860,9.4561,33.1018,32.9651,33.0189,49.2153
2000-06-06,40.50,41.56,40.44,40.88,31.792956,3843100.0,0.0,1.0,31.7930,31.7966,31.7957,28.1421,21.4543,12.1949,8.8759,32.8541,32.9651,33.0243,50.5152
2000-06-07,40.81,40.94,40.13,40.19,31.256333,3374600.0,0.0,1.0,31.6141,31.6984,31.7443,28.1915,18.0805,11.3155,8.4420,32.5475,32.9647,33.0285,46.9607
2000-06-08,40.44,41.13,40.13,40.44,31.450761,3074300.0,0.0,1.0,31.5597,31.6534,31.7164,28.2433,15.0924,10.1965,7.9010,31.8668,32.8694,33.0135,48.3710
2000-06-09,40.50,42.00,40.50,41.56,32.321801,4018700.0,0.0,1.0,31.8137,31.7749,31.7740,28.3080,18.4585,10.5543,7.9484,32.4125,32.9404,33.0494,54.2146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,172.45,175.11,171.75,174.31,174.310000,7346737.0,0.0,1.0,172.7967,174.3104,177.4579,174.3841,37.2423,27.0956,18.3145,175.0880,181.8265,190.9171,39.5286
2021-05-25,175.60,177.45,175.40,176.17,176.170000,8089738.0,0.0,1.0,173.9212,174.6485,177.3352,174.4125,34.9949,24.6543,17.6310,177.4588,179.6344,190.2630,43.8485
2021-05-26,176.62,177.90,176.20,176.47,176.470000,8086203.0,0.0,1.0,174.7708,174.9797,177.2528,174.4451,33.9676,22.2076,16.8766,178.1753,179.1448,189.7361,44.5333


Removing Columns and Making the Columns relationally based on price

In [104]:
# modifying the data to percentages as a product of price
data['EMA_5'] = ((data['EMA_5'] - data['5. adjusted close']) / data['5. adjusted close'])
data['EMA_10'] = ((data['EMA_10'] - data['5. adjusted close']) / data['5. adjusted close'])
data['EMA_20'] = ((data['EMA_20'] - data['5. adjusted close']) / data['5. adjusted close'])
data['EMA_125'] = ((data['EMA_125'] - data['5. adjusted close']) / data['5. adjusted close'])
data['BB_5 Upper Band'] = ((data['BB_5 Upper Band'] - data['5. adjusted close']) / data['5. adjusted close'])
data['BB_10 Upper Band'] = ((data['BB_10 Upper Band'] - data['5. adjusted close']) / data['5. adjusted close'])
data['BB_20 Upper Band'] = ((data['BB_20 Upper Band'] - data['5. adjusted close']) / data['5. adjusted close'])

# dropping unneeded columns
data = data.drop(columns=['1. open', '2. high', '3. low', '4. close', '7. dividend amount', '6. volume', '8. split coefficient'])

# generating our price training list based on past price action
expected_list = [0 for _ in range(len(data))]
close_list = data['5. adjusted close'].tolist()
for i in range(1, len(close_list)-3):
    three_percent = close_list[i] * 1.03
    if ((close_list[i+1] >= three_percent) or 
    (close_list[i+2] >= three_percent) or (close_list[i+3] >= three_percent)):
      expected_list[i] = 1
data = data.assign(Expected = expected_list)
expected_list


# adding the percentage change column LAST, as all dependencies have been updated already
data['5. adjusted close'] = pct_price['5. adjusted close']
data

,5. adjusted close,EMA_5,EMA_10,EMA_20,EMA_125,ADX_5,ADX_10,ADX_20,BB_5 Upper Band,BB_10 Upper Band,BB_20 Upper Band,RSI_15,Expected
date,,,,,,,,,,,,,
2000-06-05,NaN,0.006158,0.006294,0.006249,-0.111250,25.1923,13.3860,9.4561,0.047574,0.043248,0.044951,49.2153,0
2000-06-06,0.006153,0.000001,0.000115,0.000086,-0.114832,21.4543,12.1949,8.8759,0.033377,0.036868,0.038730,50.5152,0
2000-06-07,-0.016879,0.011446,0.014143,0.015612,-0.098055,18.0805,11.3155,8.4420,0.041309,0.054657,0.056698,46.9607,1
2000-06-08,0.006220,0.003464,0.006443,0.008446,-0.101984,15.0924,10.1965,7.9010,0.013228,0.045107,0.049688,48.3710,0
2000-06-09,0.027695,-0.015720,-0.016921,-0.016948,-0.124182,18.4585,10.5543,7.9484,0.002806,0.019139,0.022511,54.2146,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,0.011079,-0.008682,0.000002,0.018059,0.000425,37.2423,27.0956,18.3145,0.004463,0.043121,0.095273,39.5286,0
2021-05-25,0.010671,-0.012765,-0.008637,0.006614,-0.009976,34.9949,24.6543,17.6310,0.007316,0.019665,0.079997,43.8485,0
2021-05-26,0.001703,-0.009629,-0.008445,0.004436,-0.011474,33.9676,22.2076,16.8766,0.009663,0.015157,0.075175,44.5333,0


In [105]:
import datetime 

if len(data) > (252*3):
  tod = datetime.datetime.now()
  d = datetime.timedelta(days = 365*3)
  a = tod - d
  identifier = str(a)[:10]
  data = data[(data.index >= identifier)]

data


,5. adjusted close,EMA_5,EMA_10,EMA_20,EMA_125,ADX_5,ADX_10,ADX_20,BB_5 Upper Band,BB_10 Upper Band,BB_20 Upper Band,RSI_15,Expected
date,,,,,,,,,,,,,
2018-05-31,-0.005101,0.011330,0.019610,0.022275,0.034667,45.9082,20.0377,14.1804,0.036542,0.065067,0.060882,40.8766,0
2018-06-01,-0.001106,0.008300,0.016969,0.021177,0.035244,49.9438,21.8807,15.0994,0.029017,0.062179,0.061297,40.5092,0
2018-06-04,0.008857,-0.000369,0.006579,0.011050,0.025741,48.3794,22.6768,15.6167,0.001569,0.047189,0.052254,44.7654,0
2018-06-05,-0.002993,0.001756,0.007854,0.012743,0.028362,47.0143,23.3733,16.0916,0.005205,0.042297,0.055250,43.6254,1
2018-06-06,0.019712,-0.011739,-0.009513,-0.006183,0.008349,41.0901,21.4574,15.4960,0.001115,0.013781,0.034907,52.1916,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,0.011079,-0.008682,0.000002,0.018059,0.000425,37.2423,27.0956,18.3145,0.004463,0.043121,0.095273,39.5286,0
2021-05-25,0.010671,-0.012765,-0.008637,0.006614,-0.009976,34.9949,24.6543,17.6310,0.007316,0.019665,0.079997,43.8485,0
2021-05-26,0.001703,-0.009629,-0.008445,0.004436,-0.011474,33.9676,22.2076,16.8766,0.009663,0.015157,0.075175,44.5333,0


Applying a MinMax scaler to normalize all of the data

In [106]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

print(data)

data[["5. adjusted close",	"EMA_5",	"EMA_10",	"EMA_20",	"EMA_125", "ADX_5",	"ADX_10",	"ADX_20",	"BB_5 Upper Band",	"BB_10 Upper Band",	"BB_20 Upper Band",	"RSI_15"]] = min_max_scaler.fit_transform(data[["5. adjusted close",	"EMA_5",	"EMA_10",	"EMA_20",	"EMA_125", "ADX_5",	"ADX_10",	"ADX_20",	"BB_5 Upper Band",	"BB_10 Upper Band",	"BB_20 Upper Band",	"RSI_15"]])
data

            5. adjusted close     EMA_5  ...   RSI_15  Expected
date                                     ...                   
2018-05-31          -0.005101  0.011330  ...  40.8766         0
2018-06-01          -0.001106  0.008300  ...  40.5092         0
2018-06-04           0.008857 -0.000369  ...  44.7654         0
2018-06-05          -0.002993  0.001756  ...  43.6254         1
2018-06-06           0.019712 -0.011739  ...  52.1916         0
...                       ...       ...  ...      ...       ...
2021-05-24           0.011079 -0.008682  ...  39.5286         0
2021-05-25           0.010671 -0.012765  ...  43.8485         0
2021-05-26           0.001703 -0.009629  ...  44.5333         0
2021-05-27           0.014563 -0.015896  ...  50.1167         0
2021-05-28          -0.002178 -0.009165  ...  49.3096         0

[755 rows x 13 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,5. adjusted close,EMA_5,EMA_10,EMA_20,EMA_125,ADX_5,ADX_10,ADX_20,BB_5 Upper Band,BB_10 Upper Band,BB_20 Upper Band,RSI_15,Expected
date,,,,,,,,,,,,,
2018-05-31,0.455324,0.437003,0.411988,0.377081,0.369227,0.416969,0.187369,0.135059,0.112080,0.233238,0.171109,0.314697,0
2018-06-01,0.469907,0.422948,0.403602,0.374596,0.369974,0.467992,0.216729,0.154008,0.088936,0.226543,0.171700,0.308997,0
2018-06-04,0.506270,0.382751,0.370620,0.351674,0.357668,0.448213,0.229411,0.164675,0.004514,0.191800,0.158820,0.375026,0
2018-06-05,0.463019,0.392600,0.374669,0.355507,0.361062,0.430954,0.240507,0.174468,0.015695,0.180462,0.163087,0.357341,1
2018-06-06,0.545892,0.330019,0.319536,0.312669,0.335148,0.356052,0.209986,0.162186,0.003117,0.114368,0.134113,0.490233,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,0.514381,0.344199,0.349743,0.367540,0.324888,0.307404,0.299805,0.220304,0.013415,0.182372,0.220093,0.293784,0
2021-05-25,0.512891,0.325263,0.322320,0.341635,0.311420,0.278989,0.260914,0.206210,0.022188,0.128005,0.198334,0.360802,0
2021-05-26,0.480159,0.339807,0.322927,0.336705,0.309479,0.266001,0.221937,0.190654,0.029409,0.117557,0.191466,0.371425,0


Saving as a CSV based on the name of the ticker passed.

In [107]:
filename = "%s.csv" % ticker
data.to_csv(filename, index=False)

In [108]:
# dropping unneeded columns

dates = data.index
data.reset_index(inplace=True)
data

,date,5. adjusted close,EMA_5,EMA_10,EMA_20,EMA_125,ADX_5,ADX_10,ADX_20,BB_5 Upper Band,BB_10 Upper Band,BB_20 Upper Band,RSI_15,Expected
0,2018-05-31,0.455324,0.437003,0.411988,0.377081,0.369227,0.416969,0.187369,0.135059,0.112080,0.233238,0.171109,0.314697,0
1,2018-06-01,0.469907,0.422948,0.403602,0.374596,0.369974,0.467992,0.216729,0.154008,0.088936,0.226543,0.171700,0.308997,0
2,2018-06-04,0.506270,0.382751,0.370620,0.351674,0.357668,0.448213,0.229411,0.164675,0.004514,0.191800,0.158820,0.375026,0
3,2018-06-05,0.463019,0.392600,0.374669,0.355507,0.361062,0.430954,0.240507,0.174468,0.015695,0.180462,0.163087,0.357341,1
4,2018-06-06,0.545892,0.330019,0.319536,0.312669,0.335148,0.356052,0.209986,0.162186,0.003117,0.114368,0.134113,0.490233,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,2021-05-24,0.514381,0.344199,0.349743,0.367540,0.324888,0.307404,0.299805,0.220304,0.013415,0.182372,0.220093,0.293784,0
751,2021-05-25,0.512891,0.325263,0.322320,0.341635,0.311420,0.278989,0.260914,0.206210,0.022188,0.128005,0.198334,0.360802,0
752,2021-05-26,0.480159,0.339807,0.322927,0.336705,0.309479,0.266001,0.221937,0.190654,0.029409,0.117557,0.191466,0.371425,0
753,2021-05-27,0.527100,0.310741,0.290834,0.306222,0.291633,0.290455,0.200955,0.167700,0.018433,0.087947,0.163340,0.458044,0


# Keras NN w/Predictions

[Tutorial](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/)

In [109]:
#predict
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

#load the dataset
dataset = loadtxt(filename, delimiter=',', skiprows=2)
#split into input(x) and output(y) variables
X=dataset[:,0:12]
y=dataset[:,12]

#define the keras model
model = Sequential()
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#fit the keras model on the dataset
model.fit(X, y, epochs=300, batch_size=10)

# make class predictions with the model
predictions = model.predict_classes(X)
print(predictions)

#summarize teh first X cases
for i in range(len(X)):
  print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))


Epoch 1/300
76/76 [==============================] - 1s 1ms/step - loss: 0.6141 - accuracy: 0.8297
Epoch 2/300
76/76 [==============================] - 0s 1ms/step - loss: 0.4787 - accuracy: 0.8216
Epoch 3/300
76/76 [==============================] - 0s 2ms/step - loss: 0.4774 - accuracy: 0.8207
Epoch 4/300
76/76 [==============================] - 0s 2ms/step - loss: 0.4759 - accuracy: 0.8269
Epoch 5/300
76/76 [==============================] - 0s 1ms/step - loss: 0.4443 - accuracy: 0.8420
Epoch 6/300
76/76 [==============================] - 0s 1ms/step - loss: 0.4718 - accuracy: 0.8237
Epoch 7/300
76/76 [==============================] - 0s 2ms/step - loss: 0.4557 - accuracy: 0.8331
Epoch 8/300
76/76 [==============================] - 0s 2ms/step - loss: 0.4425 - accuracy: 0.8383
Epoch 9/300
76/76 [==============================] - 0s 2ms/step - loss: 0.4396 - accuracy: 0.8353
Epoch 10/300
76/76 [==============================] - 0s 1ms/step - loss: 0.3794 - accuracy: 0.8742
Epoch 11/

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]


In [110]:
#Change Timezone

!rm /etc/localtime
!ln -s /usr/share/zoneinfo/US/Pacific /etc/localtime
!date


Sun May 30 09:28:16 PDT 2021


In [111]:
#export the predictions to CSV
import pandas as pd
from datetime import date
import numpy

# test_DF = pd.DataFrame(predictions, columns=['prediction'])
# test_DF['date']= data['date']
# test_DF['ticker']=ticker
# test_DF = test_DF.set_index('date')
# print(test_DF)

# # generating our price prediction list
# # this was a test to see backtest results without consecutive buy signals
# prediction_list = [0 for _ in range(len(test_DF))]
# pred_list = test_DF['prediction'].tolist()
# print(pred_list)
# for i in range(1, len(pred_list)):
#     if i == 1 and ((pred_list[i-1] == 1)):
#       pred_list[i] = 0
#     if i == 2 and ((pred_list[i-2] == 1)):
#       pred_list[i] = 0
#     if ((pred_list[i-1] == 1) or 
#     (pred_list[i-2] == 1) or (pred_list[i-3] == 1)):
#       pred_list[i] = 0

# test_DF['prediction']= pred_list
# print(test_DF)

test_DF = pd.DataFrame(predictions, columns=['prediction'])
test_DF['date']= data['date']
test_DF['expected'] = data["Expected"].copy()
comparison_column = numpy.where(test_DF['prediction'] == test_DF['expected'], 1, 0)
test_DF["Equal"] = comparison_column
correctBuySignal = numpy.where((test_DF['prediction'] == test_DF['expected']) & (test_DF['Equal'] == test_DF['expected']), 1, 0)
test_DF["correctBuySignal"] = correctBuySignal
test_DF['ticker'] = ticker
test_DF = test_DF.set_index('date')
print(test_DF)

today = date.today()
saveLocation = "/content/%s_pred_%s.csv" %(ticker,today)
test_DF.to_csv(saveLocation)
#test_DF


            prediction  expected  Equal  correctBuySignal ticker
date                                                            
2018-05-31           0         0      1                 0    DIS
2018-06-01           0         0      1                 0    DIS
2018-06-04           0         0      1                 0    DIS
2018-06-05           0         1      0                 0    DIS
2018-06-06           0         0      1                 0    DIS
...                ...       ...    ...               ...    ...
2021-05-21           0         0      1                 0    DIS
2021-05-24           0         0      1                 0    DIS
2021-05-25           0         0      1                 0    DIS
2021-05-26           0         0      1                 0    DIS
2021-05-27           0         0      1                 0    DIS

[754 rows x 5 columns]
